In [1]:
# 网络中的网络(NiN)

# 全连接层的问题
#  - 卷积层需要较少的参数 c_i x c_o x k^2 
#  - 但卷积层后的第一个全连接层占了将近所有的参数, 并且会带来过拟合

# NiN块
# 一个卷积层后跟两个全连接层(1x1卷积相当于全连接层)
#  - stride是1, no padding, 输出形状跟卷积层输出一样
#  - 起到全连接层的作用
#  - 按照输入像素逐一去做的全连接层
# 结构 Convolution -> 1x1 Convolution -> 1x1 Convolution

# NiN架构
#  - 无全连接层
#  - 交替使用NiN块和步幅为2的最大池化层
#    - 逐步减少高宽和增大通道数
#  - 最后使用全局平均池化层得到输出
#    - 其输入通道数是类别数
# 这里的全局平均池化层是将每个通道的特征图'feature map', 求整个图的平均, 记为一个数字, 这样这个通道就可以被简化为一个单一个数值。
# 这个数值可以反应该通道特征在整个输入图像中的重要性。
# 最后是输入一个长为(多少类)的向量, 每个值是对这个类别的预测, 再加个softmax就是概率

# 也就是说: 假如有1000类作为输入通道, 在全局平均池化层后, 就是对每一个类的平均预测, 也就是重要性;
# 1000个类可以理解为1000个特征图, 哪个特征图的权重高, 那么对应的类可能性越大

# 结构
# - NiN块 11x11 Conv stride 4, 3x3 Maxpool stride 2
# - NiN块 5x5 Conv pad 1, 3x3 Maxpool stride 2
# - NiN块 3x3 Conv pad 1, 3x3 Maxpool stride 2
# - NiN块 3x3 Conv pad 1, Global AvgPool

# 总结
# - NiN块使用卷积层加两个1x1卷积层, 后者对每个像素增加了非线性性
# - NiN使用全局平均池化层来代替VGG和AlexNet的全连接层, 不容易过拟合, 更少的参数

In [2]:
# NiN
import torch
from torch import nn
from d2l import torch as d2l

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU())

In [4]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=1), 
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(3, stride=2), nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, stride=1, padding=1), # 10是类别的个数
    nn.AdaptiveAvgPool2d((1, 1)), # 高宽都变成1的平均池化层
    nn.Flatten()) # 最后flatten变成batch_size x 10的东西

In [5]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

training on cpu
